In [1]:
# libraries

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager as CM
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

import pandas as pd
import re
from deep_translator import GoogleTranslator
import tqdm
import os

In [2]:
# helper functions:
def myTranslate(t):
    # Translate to English:
    text = GoogleTranslator(source='auto', target='en').translate(t)
    return text
    
def extractNumber(t):
    phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)
    return phone

def getLabelData(elem, industry):
    ts = elem.text.split("\n")
    name = ts[0]
    
    # TODO 
#     name = myTranslate(name)
    
    reviews = ts[1]
    try:
        address = ts[2]
    except:
        address = ""
        
#     address = myTranslate(address)

#     temp = name + " || " + address
#     trans = myTranslate(temp)
#     try:
#         name,address = trans.split("||")
#     except:
#         print("Issue with splitting ::",trans)
#         name = myTranslate(name)
#         address = myTranslate(address)
    
    
    # try replace
    address = address.replace("Pharmaceutical company ·","")
    address = address.replace("","")
    if industry in address:
        address = address.replace(industry,"")
        
    
    try:
        timings_phone = ts[3]
    except:
        timings_phone = ""
        
    temp = extractNumber(timings_phone)
    if len(temp)>0:
        phonenumber = temp[0]
    else:
        phonenumber = ""
    # adding website
    
    try:
        a = elem.find_element(By.TAG_NAME,"a")
        website = a.get_attribute('href')
    except:
        website = ""
        
    obj = {
        "name":name.strip(),
        "phone":phonenumber,
        "website":website,
        "address":address.strip()
        
    }
    return obj


In [ ]:
# Initiating the browser : Old way
options = webdriver.ChromeOptions()
path = "../chrome/chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

In [3]:
# New way :>
chrome_options = Options()
# Use WebDriverManager to handle driver installation
service = Service(executable_path=CM().install())
browser = webdriver.Chrome(service=service, options=chrome_options)

In [4]:
browser.maximize_window()

In [5]:
browser.get_window_size()

{'width': 1552, 'height': 832}

### Step 1: Industry Selection

In [5]:
industry = "Medical shop"

### 1. High level search:

In [6]:
country = "Spain" # Please change accordingly

In [7]:
company_category = f"{industry} Companies"
company_category = f"{industry}"
url = "https://www.google.com/maps/search/{} in {}".format(company_category, country)
company_category, url

('Medical shop', 'https://www.google.com/maps/search/Medical shop in Spain')

In [14]:
data = []

In [15]:
# started extracting
print("Currently extracting from :: ",url)

# Open the Google Map URL
browser.get(url)
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80 # If you get "target move out of boundaries" => change this value
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)




Currently extracting from ::  https://www.google.com/maps/search/Medical shop in Spain
28 times reached !!


In [16]:
# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem,industry)
    data.append(d)


In [22]:
## saving results:
df = pd.DataFrame(data)
# df.head()
df

,name,phone,website,address
0,Comercial Médico Quirúrgica,,,Tienda de material sanitario · · Carrer de Ca...
1,Farmacia Guillermo Chacón Puig,934 35 92 93,http://farmaciaguillermochacon.com/,"Farmacia · · Carrer de la Indústria, 128"
2,Farmàcia Dra. Pascal,934 35 82 68,,"Farmacia · · AL COSTAT HOSP. DE SANT PAU, Av...."
3,Farmacia Gaudi - Maria Belenguer,934 35 80 90,,"Farmacia · · Av. de Gaudí, 71"
4,Ambi Medical,,,Tienda de material sanitario · · Carrer de Sa...
...,...,...,...,...
115,Farmacia Guito Vilardell,933 17 74 91,http://www.farmaciabruc64.es/contactar/farmaci...,"Farmacia · · Carrer del Bruc, 64"
116,BCNPharma,933 19 85 07,https://bcnpharma.com/,"Farmacia · · Plaça de Sant Agustí Vell, 11"
117,Farmàcia Lda Mercedes Rojo,934 58 87 26,,"Farmacia · · Carrer de la Indústria, 65"
118,FARMACIA YOLANDA GIMÉNEZ CHANCHO,932 13 88 96,,"Farmacia · Carrer de Romans, 1"


In [17]:
df.shape

(100, 4)

In [19]:
# creating a folder
os.mkdir("../results/industries/{}".format(industry))

In [20]:
os.mkdir("../results/industries/{}/{}".format(industry,country))

In [21]:
df.to_csv("../results/industries/{}/{}/companies_highlevel_1.csv".format(industry,country),index=None)

#### Cites wise

In [14]:
# loading the list of cities
cities_df = pd.read_csv("../data/worldcities.csv")
cities = list(cities_df[cities_df['country'] == country]['city_ascii'].values)

In [15]:
len(cities)

754

In [16]:
cities

['Madrid',
 'Barcelona',
 'Sevilla',
 'Malaga',
 'Valencia',
 'Bilbao',
 'Zaragoza',
 'Murcia',
 'Palma',
 'Las Palmas',
 'Alicante',
 'Cordoba',
 'Valladolid',
 'Vigo',
 'Gijon',
 'Vitoria-Gasteiz',
 'A Coruna',
 'Elche',
 'Granada',
 'Badalona',
 'Tarrasa',
 'Sabadell',
 'Petrel',
 'Oviedo',
 'Cartagena',
 'Mostoles',
 'Jerez de la Frontera',
 'Pamplona',
 'Santa Cruz',
 'Almeria',
 'Alcala de Henares',
 'Fuenlabrada',
 'Leganes',
 'Donostia',
 'Getafe',
 'Santander',
 'Burgos',
 'Alcorcon',
 'Castellon de la Plana',
 'Albacete',
 'Guadalajara',
 'La Laguna',
 'Logrono',
 'Marbella',
 'Badajoz',
 'Salamanca',
 'Huelva',
 'Lleida',
 'Dos Hermanas',
 'Tarragona',
 'Parla',
 'Torrejon de Ardoz',
 'Mataro',
 'Leon',
 'Algeciras',
 'Cadiz',
 'Alcobendas',
 'Jaen',
 'Reus',
 'Ourense',
 'Girona',
 'Roquetas de Mar',
 'San Vicente de Baracaldo',
 'Telde',
 'Santiago de Compostela',
 'San Fernando',
 'Ciudad de Melilla',
 'Fuengirola',
 'Ciudad de Ceuta',
 'San Baudilio de Llobregat',
 'Tole

In [23]:
## Remember to drag , zoom in and then extract : repeat
## Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome/chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

# browser.set_page_load_timeout(300) # only for 5 minutes

In [ ]:
for city in tqdm.tqdm(cities[:10]):
    data = []
    url = "https://www.google.com/maps/search/{} companies in {} {}".format(industry, city, country)
    url = "https://www.google.com/maps/search/{} in {} {}".format(industry, city, country)
#     print("Currently extracting from :: ",url)
    browser.get(url)
    
    # scrolling : 
    actionChains = ActionChains(browser)

    # scrolling upto 20 times::
    end = False
    repeat = 0
    limit = 28

    while not end and repeat < limit:

        # check end
        end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
        if len(end_identify)>0:
            end = True
#             print("Reached end !!")

        repeat += 1
        if repeat == limit:
#             print("28 times reached !!")
            end = True

        elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
        if len(elems)==0:
#             print("Remember to add ")
            continue
        elem = elems[-1]

        xoffset = 0
        yoffset = 80

        actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
        browser.execute_script("arguments[0].scrollIntoView();", elem)
        time.sleep(0.2)


    # extracting the data
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    for elem in elems:
        d = getLabelData(elem,industry)
        data.append(d)

#     print("Done")
#     print("Extracted :: ",len(elems))
    
#     print("Saving the file:")
    
    df = pd.DataFrame(data)
    df.to_csv("../results/industries/{}/{}/{}.csv".format(industry,country,city),index=None)
    

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████            | 9/10 [04:21<00:30, 30.28s/it]

In [ ]:
# Note: There are times when you get Aw, Swap error from google maps: then do below
# Check the number of iteration done till now 
# continue iterating cities from that city
# Run again

In [ ]:
## Saving all in one file :> 

In [34]:
import pandas as pd
import os

In [35]:
df = pd.DataFrame()

In [36]:
files = os.listdir("../results/industries/Software/Spain")

In [37]:
for f in files:
    try:
        temp = pd.read_csv("../results/industries/Software/Spain/" + f)
        # Adding city to the dataframe
        temp_records = temp.to_dict("records")
        temp_records_updated = []
        for o in temp_records:
            o.update({"city":f.split(".")[0]})
            temp_records_updated.append(o)
        temp = pd.DataFrame(temp_records_updated)

        df = pd.concat([df,temp])
        df = df.drop_duplicates()
    except:
        print(f)

Amorebieta.csv
Caspe.csv
Fortuna.csv
Moana.csv
Mora.csv
Mula.csv
Osuna.csv
Pulpi.csv
Ripoll.csv
Santa Margarita de Mombuy.csv
Sarria.csv
Sevilla La Nueva.csv
Teguise.csv
Yuncos.csv


In [38]:
df.shape

(30661, 5)

In [39]:
df.to_csv("../results/industries/Software/spain.csv", index=None)

#### Selected City - Do Zoom in | Drag Drop | Extract || Repeat

In [23]:
data = []

In [24]:
city = "Riyadh" # Mostly do it for capital city
url = "https://www.google.com/maps/search/Pharmaceutical companies in {} {}".format(city,country)
#     print("Currently extracting from :: ",url)
browser.get(url)

In [25]:
## zoom in 
zoom_button = browser.find_element("xpath",'.//button[@id="widget-zoom-in"]')
action = ActionChains(browser)
action.click(on_element = zoom_button)
action.perform() 

In [26]:
## search this area
search_this_area = browser.find_element("xpath",'.//span[@class="Kw7Rxf"]')
action.click(on_element = search_this_area)
action.perform() 

In [28]:
## Extracting again
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)
    time.sleep(0.2)


# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem,industry)
    data.append(d)
    
print("Done")
print("Extracted :: ",len(elems))

Reached end !!
Done
Extracted ::  5


In [29]:
# filtering duplicates:
## removing duplicates
final_data = []
for d in data:
    if d not in final_data:
        final_data.append(d)

In [30]:
len(final_data)

21

In [31]:
df = pd.DataFrame(final_data)

In [32]:
df.to_csv("../results/names/{}/Companies_Rest_All.csv".format(country,city),index=None)

#### City Wise all Category of industries

In [15]:
categories = [
    "Software Companies",
    "Pharma Companies",
    "Industries",
    "Manufacturing companies",
    "Small and medium scale industries",
    "Food Industries",
    "Tourism companies",
    "Tourist industries"
]

In [16]:
city = "Barcelona"
country = "Spain"

In [17]:
data = []
for category in tqdm.tqdm(categories):
    category_data = []
    url = "https://www.google.com/maps/search/{} in {} {}".format(category, city, country)
    browser.get(url)
    
    # scrolling : 
    actionChains = ActionChains(browser)

    # scrolling upto 20 times::
    end = False
    repeat = 0
    limit = 28

    while not end and repeat < limit:

        # check end
        end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
        if len(end_identify)>0:
            end = True
#             print("Reached end !!")

        repeat += 1
        if repeat == limit:
#             print("28 times reached !!")
            end = True

        elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
        if len(elems)==0:
#             print("Remember to add ")
            continue
        elem = elems[-1]

        xoffset = 0
        yoffset = 80

        actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
        browser.execute_script("arguments[0].scrollIntoView();", elem)
        time.sleep(0.2)


    # extracting the data
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    for elem in elems:
        d = getLabelData(elem,category)
        d['industry'] = category
        data.append(d)
        category_data.append(d)
    category_data_df = pd.DataFrame(category_data)
    category_data_df.to_csv("../results/barcelona/{}.csv".format(category),index=None)
    
df = pd.DataFrame(data)
df = df.drop_duplicates()
df.to_csv("../results/barcelona/all_categories.csv",index=None)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\yeshu\AppData\Local\Temp\ipykernel_10812\224092416.py:8: FutureWarning: Possible nested set at position 1
  phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:10<00:00, 16.26s/it]
